This notebook analyses the differences between the sexes by age in Ireland, from the following link: 
"https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
- Weighted mean age (by sex).
- The difference between the sexes by age.

In [19]:
import pandas as pd

In [20]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)
df.tail(3)

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
9789,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-149d-13a3-e055-000000000001,Cavan County Council,Number,12
9790,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-14a4-13a3-e055-000000000001,Donegal County Council,Number,31
9791,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-1495-13a3-e055-000000000001,Monaghan County Council,Number,7


Drop columns that we don't need.

In [21]:
# Check name of columns
header = df.columns.tolist()
print(header)

['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'CensusYear', 'C02199V02655', 'Sex', 'C02076V03371', 'Single Year of Age', 'C03789V04537', 'Administrative Counties', 'UNIT', 'VALUE']


In [22]:
drop_col_list = ['STATISTIC','Statistic Label', 'TLIST(A1)', 'CensusYear','C02199V02655', 'C02076V03371','C03789V04537','UNIT']
df.drop(columns=drop_col_list, inplace=True) # Inplace=True to modify the DataFrame directly
print(df.head(3))  # print the first 3 rows to verify

          Sex Single Year of Age Administrative Counties    VALUE
0  Both sexes           All ages                 Ireland  5149139
1  Both sexes           All ages   Carlow County Council    61968
2  Both sexes           All ages     Dublin City Council   592713


There's data for each gender and we need to get rid of 'Both sexes'

In [23]:
df = df[df["Sex"] != "Both sexes"]
print(df.tail(3))  # print the last 3 rows to verify

         Sex  Single Year of Age  Administrative Counties  VALUE
9789  Female  100 years and over     Cavan County Council     12
9790  Female  100 years and over   Donegal County Council     31
9791  Female  100 years and over  Monaghan County Council      7


Having words in the age field, such as "All ages," "under 1 year," "years and over," and "years," can be problematic when analyzing the data. Therefore, we will remove these words and keep only the age as a number.

First step is remove the "All ages" text.

In [24]:
df = df[df["Single Year of Age"] != "All ages"]
print(df.head(3))  # print the first 3 rows to verify

       Sex Single Year of Age Administrative Counties  VALUE
3296  Male       Under 1 year                 Ireland  29610
3297  Male       Under 1 year   Carlow County Council    346
3298  Male       Under 1 year     Dublin City Council   3188


Second step is change the column using find and replace. We will replace everything that says "Under 1 year" for "0" consider the minors have less than 1 year.

In [25]:
df["Single Year of Age"] = df["Single Year of Age"].replace("Under 1 year", "0")
print(df.head(5))  # print the first 5 rows to verify

       Sex Single Year of Age                Administrative Counties  VALUE
3296  Male                  0                                Ireland  29610
3297  Male                  0                  Carlow County Council    346
3298  Male                  0                    Dublin City Council   3188
3299  Male                  0  Dún Laoghaire Rathdown County Council   1269
3300  Male                  0                  Fingal County Council   2059


Third step is change the text that is repeated regulary in the data set "years". To avoid do this one by one: '2 years' '3 years', we can use regular expressions, that they help to change/delete large amounts of data that are repeated in the data set.

Source: https://www.w3schools.com/python/python_regex.asp

In [26]:
# Regular expression to remove 'years' and 'year' from the 'Single Year of Age' column
df["Single Year of Age"] = df["Single Year of Age"].str.replace(r'\D', '', regex=True) # add r to indicate raw string
print(df.tail(5))  # print the last 5 rows to verify

         Sex Single Year of Age   Administrative Counties  VALUE
9787  Female                100  Roscommon County Council      7
9788  Female                100      Sligo County Council      9
9789  Female                100      Cavan County Council     12
9790  Female                100    Donegal County Council     31
9791  Female                100   Monaghan County Council      7


In [27]:
#Create a new CSV file with the cleaned data
df.to_csv("population_for_analysis.csv")

In [28]:
print(df.info())  # print DataFrame info to verify data types

<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 3296 to 9791
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      6464 non-null   object
 1   Single Year of Age       6464 non-null   object
 2   Administrative Counties  6464 non-null   object
 3   VALUE                    6464 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 252.5+ KB
None


As we can see the Single Year of age column is an object and we want to have this values as a integer. We can change this using 'type' to 'int 64' 

In [29]:
df["Single Year of Age"] = df["Single Year of Age"].astype('int64') # Convert to integer

In [30]:
print(df.info())  # print DataFrame info to verify data types

<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 3296 to 9791
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      6464 non-null   object
 1   Single Year of Age       6464 non-null   int64 
 2   Administrative Counties  6464 non-null   object
 3   VALUE                    6464 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 252.5+ KB
None


## Pivot Tables
We can use pivot tables to summarize, analyze, and explore large datasets in a organized way.
And avoid certain issues when weuse cross tables. 


Sources:
https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html
https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

In [31]:
# Create pivot table, to get the sum of population by age and sex
df_analysis = pd.pivot_table(df, index ='Single Year of Age', columns='Sex', values='VALUE', aggfunc='sum') 
#aggfunc='sum' to sum the values
print(df_analysis.head(10))  # print the first 10 rows to verify
df_analysis.to_csv("population_for_analysis.csv")

Sex                 Female   Male
Single Year of Age               
0                    56372  59220
1                    55090  57750
2                    57948  60472
3                    58966  62002
4                    59638  63372
5                    62684  65368
6                    65244  68184
7                    67144  70858
8                    68874  72592
9                    70454  73938


# Weighted mean age (by sex)

In [32]:
headers = list(df_analysis.columns)
genders = headers[0:]  # Get all genders from the columns
print(genders)

['Female', 'Male']


In [33]:
age_female = df_analysis[genders[1]].sum()
print(age_female)

5089098


In [35]:
print(df_analysis) 


Sex                 Female   Male
Single Year of Age               
0                    56372  59220
1                    55090  57750
2                    57948  60472
3                    58966  62002
4                    59638  63372
...                    ...    ...
96                    1912    654
97                    1464    434
98                     984    260
99                     672    210
100                   1168    308

[101 rows x 2 columns]
